In [1]:
import subprocess
import re
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
from collections import defaultdict
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import defaultdict


sns.set(style="whitegrid")

![Image](bert-like-before.png) 

![Image](bert-like-after.png) 

In [2]:
configs = [
    ("DD", "0"),
    ("SparseDense", "0"),
    ("SparseDense", "1"),
]
sparsities = ["0.10", "0.3", "0.5", "0.7", "0.9"]

repeats = 10
binary = "../../build/benchmark"
graph_name = "bert"

In [3]:
def parse_output(output):
    metrics = {}
    lines = output.strip().splitlines()
    for line in lines:
        if "analysis" in line:
            metrics["analysis"] = float(line.split("=")[-1].strip())
        elif "load graph" in line:
            metrics["load"] = float(line.split("=")[-1].strip())
        elif "compilation" in line:
            metrics["compilation"] = float(line.split("=")[-1].strip())
        elif "runtime" in line:
            metrics["runtime"] = float(line.split("=")[-1].strip())
        elif "memory used" in line:
            metrics["memory"] = float(line.split("=")[-1].strip())
        elif "before" in line:
            metrics["before"] = float(line.split("=")[-1].strip())
        elif "after" in line:
            metrics["after"] = float(line.split("=")[-1].strip())
    return metrics

def run():
    results = []
    with open(f"{graph_name}-{datetime.now()}.out", "wt") as file:
        file.write("model, row, col, format, prop, before, after, analysis, load, comp, run, memory\n")
        for fmt, opt in configs:
            for sparsity in sparsities:
                print(f"{fmt}, {opt}, {sparsity}",end='\r')
                times = {"analysis": [], "load": [], "compilation": [], "runtime": [], "memory": []}
                for _ in range(repeats):
                    cmd = [binary, "graph", graph_name, sparsity, sparsity, fmt, opt]
                    result = subprocess.run(cmd, capture_output=True, text=True)
                    metrics = parse_output(result.stdout)
                    for k in times:
                        times[k].append(metrics.get(k, 0.0))
        
                mean_metrics = {k: statistics.mean(times[k]) for k in times}
                mean_metrics["config"] = f"{sparsity},{sparsity},{fmt},opt={opt}"
                file.write(f'{graph_name},{sparsity}, {sparsity}, {fmt}, {opt}, {metrics["before"]}, {metrics["after"]}, {mean_metrics["analysis"]}, {mean_metrics["load"]}, {mean_metrics["compilation"]}, {mean_metrics["runtime"]}, {mean_metrics["memory"]}\n')
                results.append(mean_metrics)
    return results

In [4]:
def plot(data):
    # === Legend mapping ===
    legend_map = {
        "SparseDense,opt=1": "sparse+prop",
        "SparseDense,opt=0": "sparse",
        "DD,opt=0": "dense",
    }

    # === Organize data ===
    configs = defaultdict(lambda: {"ratios": [], "runtime": [], "memory": [], "analysis": [], "compilation": []})
    ratio_labels = []

    for entry in data:
        # Extract ratio and config name
        parts = entry["config"].split(",")
        ratio = f"({parts[0]},{parts[1]})"
        config_type = ",".join(parts[2:])  # e.g., "DD,opt=0"
        label = legend_map.get(config_type, config_type)

        if ratio not in ratio_labels:
            ratio_labels.append(ratio)

        configs[label]["ratios"].append(ratio)
        configs[label]["runtime"].append(entry["runtime"])
        configs[label]["memory"].append(entry["memory"])
        configs[label]["analysis"].append(entry["analysis"])
        configs[label]["compilation"].append(entry["compilation"])

    # Sort ratio_labels (by numeric value)
    def ratio_key(r):
        vals = r.strip("()").split(",")
        return float(vals[0]), float(vals[1])
    ratio_labels = sorted(ratio_labels, key=ratio_key)

    # Create index for x-axis
    x = np.arange(len(ratio_labels))

    # === 1) Runtime plot ===
    plt.figure(figsize=(8, 5))
    for label, values in configs.items():
        # Align runtime values according to ratio_labels order
        runtime_values = [v for _, v in sorted(zip(values["ratios"], values["runtime"]), key=lambda x: ratio_key(x[0]))]
        plt.plot(x, runtime_values, label=label)
    plt.xticks(x, ratio_labels, rotation=45)
    plt.ylabel("Runtime (s)")
    plt.title("Runtime vs Ratios")
    plt.legend()
    plt.tight_layout()
    #plt.savefig("runtime_plot.png")
    #plt.close()
    plt.plot()

    # === 2) Memory plot ===
    plt.figure(figsize=(8, 5))
    for label, values in configs.items():
        memory_values = [v for _, v in sorted(zip(values["ratios"], values["memory"]), key=lambda x: ratio_key(x[0]))]
        plt.plot(x, memory_values, label=label)
    plt.xticks(x, ratio_labels, rotation=45)
    plt.ylabel("Memory (MB)")
    plt.title("Memory Usage vs Ratios")
    plt.legend()
    plt.tight_layout()
    #plt.savefig("memory_plot.png")
    plt.plot()
    #plt.close()

    # === 3) Analysis vs Compilation vs Runtime (bar chart) ===
    width = 0.25
    bar_positions = np.arange(len(ratio_labels))
    
    for label, values in configs.items():
        if label != "sparse+prop":
            continue
        analysis_values = [v for _, v in sorted(zip(values["ratios"], values["analysis"]), key=lambda x: ratio_key(x[0]))]
        comp_values = [v for _, v in sorted(zip(values["ratios"], values["compilation"]), key=lambda x: ratio_key(x[0]))]
        runtime_values = [v for _, v in sorted(zip(values["ratios"], values["runtime"]), key=lambda x: ratio_key(x[0]))]

        plt.figure(figsize=(10, 6))
        plt.bar(bar_positions - width, analysis_values, width, label="analysis")
        plt.bar(bar_positions, comp_values, width, label="compilation")
        plt.bar(bar_positions + width, runtime_values, width, label="runtime")
        plt.xticks(bar_positions, ratio_labels, rotation=45)
        plt.ylabel("Time (s)")
        plt.title(f"Analysis vs Compilation vs Runtime ({label})")
        plt.legend()
        plt.tight_layout()
        #plt.savefig(f"time_breakdown_{label.replace('+','_')}.png")
        #plt.close()
        plt.plot()

In [14]:
def plotly(data):
    legend_map = {
        "SparseDense,opt=1": "sparse+prop",
        "SparseDense,opt=0": "sparse",
        "DD,opt=0": "dense",
    }

    configs = defaultdict(lambda: {"ratios": [], "runtime": [], "memory": [], "analysis": [], "compilation": []})
    ratio_labels = []

    for entry in data:
        parts = entry["config"].split(",")
        ratio = f"({parts[0]},{parts[1]})"
        config_type = ",".join(parts[2:])
        label = legend_map.get(config_type, config_type)

        if ratio not in ratio_labels:
            ratio_labels.append(ratio)

        configs[label]["ratios"].append(ratio)
        configs[label]["runtime"].append(entry["runtime"])
        configs[label]["memory"].append(entry["memory"])
        configs[label]["analysis"].append(entry["analysis"])
        configs[label]["compilation"].append(entry["compilation"])

    def ratio_key(r):
        vals = r.strip("()").split(",")
        return float(vals[0]), float(vals[1])
    ratio_labels = sorted(ratio_labels, key=ratio_key)

    # Create subplots
    fig = make_subplots(rows=3, cols=1,
                        shared_xaxes=False,
                        vertical_spacing=0.15,
                        subplot_titles=("Runtime vs Sparsity Ratio",
                                        "Memory Usage vs Sparsity Ratio",
                                        "Analysis vs Compilation vs Runtime"))
    fig.layout.template = "plotly_white"
    # === 1) Runtime Plot (row 1) ===
    for label, values in configs.items():
        runtime_values = [v for _, v in sorted(zip(values["ratios"], values["runtime"]), key=lambda x: ratio_key(x[0]))]
        fig.add_trace(
            go.Scatter(x=ratio_labels, y=runtime_values, mode="lines", name=f"Runtime ({label})", legendgroup="runtime"),
            row=1, col=1
        )

    # === 2) Memory Plot (row 2) ===
    for label, values in configs.items():
        memory_values = [v for _, v in sorted(zip(values["ratios"], values["memory"]), key=lambda x: ratio_key(x[0]))]
        fig.add_trace(
            go.Scatter(x=ratio_labels, y=memory_values, mode="lines", name=f"Memory ({label})", legendgroup="memory"),
            row=2, col=1
        )

    # === 3) Analysis vs Compilation vs Runtime (Bars) (row 3) ===
    # To avoid clutter, show aggregated bars (sum over configs)
    for label, values in configs.items():
        if label != "sparse+prop":
            continue
        analysis_values = [v for _, v in sorted(zip(values["ratios"], values["analysis"]), key=lambda x: ratio_key(x[0]))]
        comp_values = [v for _, v in sorted(zip(values["ratios"], values["compilation"]), key=lambda x: ratio_key(x[0]))]
        runtime_values = [v for _, v in sorted(zip(values["ratios"], values["runtime"]), key=lambda x: ratio_key(x[0]))]

        fig.update_yaxes(type="log", row=3, col=1)

        fig.add_trace(
            go.Bar(name=f"Analysis ({label})", x=ratio_labels, y=analysis_values, legendgroup="bar"),
            row=3, col=1
        )
        fig.add_trace(
            go.Bar(name=f"Compilation ({label})", x=ratio_labels, y=comp_values, legendgroup="bar"),
            row=3, col=1
        )
        fig.add_trace(
            go.Bar(name=f"Runtime ({label})", x=ratio_labels, y=runtime_values, legendgroup="bar"),
            row=3, col=1
        )

    # Layout adjustments
    fig.update_layout(
        height=1000,
        title_text="Benchmark Results",
        barmode='group',
        legend_tracegroupgap=300
    )
    fig.update_xaxes(title_text="Row/Col Sparsity Ratio", row=1, col=1)
    fig.update_xaxes(title_text="Row/Col Sparsity Ratio", row=2, col=1)
    fig.update_xaxes(title_text="Row/Col Sparsity Ratio", row=3, col=1)
    fig.update_yaxes(title_text="Runtime (s)", row=1, col=1)
    fig.update_yaxes(title_text="Memory (MB)", row=2, col=1)
    fig.update_yaxes(title_text="Time (s)", row=3, col=1)

    fig.show()


In [6]:
data = run()

SparseDense, 1, 0.90

In [16]:
plotly(data)

defaultdict(<function plotly.<locals>.<lambda> at 0x13a926e80>, {'dense': {'ratios': ['(0.10,0.10)', '(0.3,0.3)', '(0.5,0.5)', '(0.7,0.7)', '(0.9,0.9)'], 'runtime': [4.587369, 4.562023, 4.818735, 4.515778, 5.029347], 'memory': [487.0797, 441.1655, 315.7969, 259.4361, 178.0781], 'analysis': [0.0, 0.0, 0.0, 0.0, 0.0], 'compilation': [0.677223, 0.6874103, 0.7536917, 0.6536853, 0.7031916]}, 'sparse': {'ratios': ['(0.10,0.10)', '(0.3,0.3)', '(0.5,0.5)', '(0.7,0.7)', '(0.9,0.9)'], 'runtime': [4.185098, 3.054102, 1.901804, 1.102102, 0.3475263], 'memory': [410.0953, 342.1781, 274.5545, 208.5438, 112.19207], 'analysis': [0.0, 0.0, 0.0, 0.0, 0.0], 'compilation': [0.6804229, 0.6749955, 0.6629151, 0.7035075, 0.6790058]}, 'sparse+prop': {'ratios': ['(0.10,0.10)', '(0.3,0.3)', '(0.5,0.5)', '(0.7,0.7)', '(0.9,0.9)'], 'runtime': [6.366574, 2.399651, 1.12362, 0.4214992, 0.08711652], 'memory': [424.1985, 255.6421, 180.4734, 88.34374, 27.998450000000002], 'analysis': [1.75999e-05, 1.84666e-05, 1.71376e-0

[{'analysis': 0.0,
  'load': 2.763706,
  'compilation': 0.677223,
  'runtime': 4.587369,
  'memory': 487.0797,
  'config': '0.10,0.10,DD,opt=0'},
 {'analysis': 0.0,
  'load': 2.433844,
  'compilation': 0.6874103,
  'runtime': 4.562023,
  'memory': 441.1655,
  'config': '0.3,0.3,DD,opt=0'},
 {'analysis': 0.0,
  'load': 2.297662,
  'compilation': 0.7536917,
  'runtime': 4.818735,
  'memory': 315.7969,
  'config': '0.5,0.5,DD,opt=0'},
 {'analysis': 0.0,
  'load': 1.846614,
  'compilation': 0.6536853,
  'runtime': 4.515778,
  'memory': 259.4361,
  'config': '0.7,0.7,DD,opt=0'},
 {'analysis': 0.0,
  'load': 1.636056,
  'compilation': 0.7031916,
  'runtime': 5.029347,
  'memory': 178.0781,
  'config': '0.9,0.9,DD,opt=0'},
 {'analysis': 0.0,
  'load': 2.72876,
  'compilation': 0.6804229,
  'runtime': 4.185098,
  'memory': 410.0953,
  'config': '0.10,0.10,SparseDense,opt=0'},
 {'analysis': 0.0,
  'load': 2.547565,
  'compilation': 0.6749955,
  'runtime': 3.054102,
  'memory': 342.1781,
  'conf